## This is the code from our last project, this still needs to be updatet to our data !

In [2]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from fredapi import Fred

ek.set_app_key("320e7257c23945729695a106cedb2b9cb72d3207") # Elena
#ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8") # Ginevra

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

# Variables

**TR.RevenueActValue.date** - prints the datetime

## Part 1: Accuracy

1. **TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

2. **TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> this is a analyst forecast variable

3. **TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.

--> forecast error between actual EPS and TR.EPSMean

## Part 2: Factors

### Firm charateristics

1. **TR.RevenueActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Revenue (or Sales) is a corporation's net revenue, generally derived from core business activities. For non-financial companies, the calculation of net revenue (or net turnover) in most markets generally involves subtracting transportation and related operational costs from gross revenue/sales. Revenue recognition practices vary significantly from market to market, though generally the recording of revenue is based upon sales invoices issued (or anticipated for forecast purposes) during the accounting period.
2. **TR.F.EV** - Enterprise Value is the Company's total economic value OR the theoretical takeover price, often used as an alternative to market capitalization. Enterprise Value represents Market Capitalization TR.F.MktCap + Debt including Preferred Equity & Minority Interest - Total [TR.F.DebtlnclPrefEqMinlntrTot] (-) Cash & Short Term Investments - Total [TR.F.CashSTlnvstTot]. Market Capitalization [TR.F.MktCap] is required. The data item is calculated for all periodicities. It is applicable to all industries.
3. **TR.TotalReturn3Mo** - The 3 months total return incorporates the price Change and any relevant dividends for the last 3 months.
4. **TR.EVTosales** - EV represents the sum of Market Capitalization, Total Debt, Preferred Stock and Minority Interest minus Cash and Short Term Investments for the most recent fiscal period. Market Cap is calculated by multiplying Current Total Shares Outstanding by Latest Close Price. Sales is LTM Total Revenue. EV to Sales ratio is not calculated when LTM Sales is less than or equal to Zero.
5. **TR.F.MktCap** Market Cap
6. **TR.F.NetIncAfterTax** Net Income after Tax [SIAT] represents the income/expense after all operating and non-operating income and expense, reserves, income taxes, but before equity in earnings, minority interest, extraordinary items, after-tax adjustments, discontinued operations and preferred dividends. Applicable to all Industries. Net Income after Tax [SIAT] includes: • Net Income after Tax [XIAT]
    **loss firm status** - -1 for profit firm, +1 for loss firm based on net income after tax
7. **TR.Volume** - Volume for the latest trading day. For stock exchanges that trade share by share, Volume is number of shares that traded on the trade date. For stock exchanges that trade in lots, Volume is divided by Lotsize, if the Lotsize is greater than one.

### Analysts
1. **TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.
2. **TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).
3. change in recommendation
4. **TR.EPSCoefficientOfVariation**
5. **TR.priceTargetStdDev** The statistical standard deviation of all estimates included in the summary calculation- Price Target is the projected price level forecasted by the analyst Within a specific time nonzon-

### Macro characteristics

1. **TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.



### Return Potential

1. **TR.PriceTargetMean** The statistical average of all broker estimates determined to be on the majority accounting basis. Price target is the projected price level forecasted by the analyst within a specific time horizon.

1. **TR.PriceClose** The latest available closing price. If there are no trades for the most recent completed tradable day, the most recent prior tradable day with trading activity is used, provided the last tradable day for the instrument is within 378 completed calendar days.

### More difficult Factors (yearly):

#### ESG

1. **TR.TRESGScore** Refinitiv ESG Score is an overall company score based on the self-reported information in the environmental, social and corporate governance pillars.

#### Proportion of independent Directors

1. **TR.AnalyticIndepBoard** - Independent Board Members - Percentage of independent board members as reported by the company.

2. **TR.GovernancePillarScore** - Governance Pillar Score - The corporate governance pillar measures a company's systems and processes, which ensure that its board members and executives act in the best interests of its long term shareholders. It reflects a company's capacity, through its use of best management practices, to direct and control its rights and responsibilities through the creation of incentives, as well as checks and balances in order to generate long term shareholder value.

## Part 3: Regression

### continue..

In [3]:
economic_story = ["TR.PriceCloseDate", "TR.PriceClose", "TR.Volume", "TR.CompanyMarketCapitalization", "TR.PriceToSalesPerShare", "TR.WACCBeta", "TR.EPSActValue", "TR.PriceToBVPerShare", "TR.FwdPriceToBVPerShare", "TR.PE", "TR.TotalDebtToEV", "TR.TotalReturn", "TR.NUMBEROFTRADES", "TR.CombinedAlphaSectorRank", "TR.CombinedAlphaIndustryRank"]

# Whole Dataframe

In [4]:
df, e = ek.get_data('0#.SPX', economic_story, parameters={'SDate': '2023-05-01', 'EDate':'2023-04-01', 'Period':'FQ0', 'Frq':'D'})

In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df

,Instrument,Date,Price Close,Volume,Company Market Capitalization,Price To Sales Per Share (Daily Time Series Ratio),Beta,Earnings Per Share - Actual,Price To Book Value Per Share (Daily Time Series Ratio),Forward Price To Book Value Per Share (Daily Time Series Ratio),P/E (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Total Return,Number of Trades,Combined Alpha Model Sector Rank,Combined Alpha Model Industry Rank
0,POOL.OQ,2023-05-01 00:00:00+00:00,346.78,118561,13537684335.0,2.266157,0.932675,2.58,10.770428,8.691228,20.519587,9.1803,-1.292269,4057,15,39
1,POOL.OQ,2023-04-28 00:00:00+00:00,351.32,151372,13714917990.0,2.295825,0.932581,2.58,10.911433,8.805013,20.788227,9.07222,2.69812,4655,11,28
2,POOL.OQ,2023-04-27 00:00:00+00:00,342.09,134656,13354594942.5,2.161033,0.932581,2.58,10.820371,8.573684,18.23804,9.436726,2.961625,4380,11,28
3,POOL.OQ,2023-04-26 00:00:00+00:00,332.25,126815,12970458562.5,2.098872,0.932389,2.58,10.50913,8.327068,17.713434,9.690015,-1.386086,4486,12,34
4,POOL.OQ,2023-04-25 00:00:00+00:00,336.92,151432,13152767190.0,2.128374,0.93248,2.58,10.656843,8.44411,17.962409,9.568132,-3.80036,4842,10,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15588,AVY.N,NaT,<NA>,<NA>,<NA>,<NA>,0.963384,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15589,AVY.N,NaT,<NA>,<NA>,<NA>,<NA>,0.963342,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15590,AVY.N,NaT,<NA>,<NA>,<NA>,<NA>,0.962527,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15591,AVY.N,NaT,<NA>,<NA>,<NA>,<NA>,0.968965,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [6]:
df.head(40)

,Instrument,Date,Price Close,Volume,Company Market Capitalization,Price To Sales Per Share (Daily Time Series Ratio),Beta,Earnings Per Share - Actual,Price To Book Value Per Share (Daily Time Series Ratio),Forward Price To Book Value Per Share (Daily Time Series Ratio),P/E (Daily Time Series Ratio),Total Debt To Enterprise Value (Daily Time Series Ratio),Total Return,Number of Trades,Combined Alpha Model Sector Rank,Combined Alpha Model Industry Rank
0,POOL.OQ,2023-05-01 00:00:00+00:00,346.78,118561,13537684335.0,2.266157,0.932675,2.58,10.770428,8.691228,20.519587,9.1803,-1.292269,4057,15,39
1,POOL.OQ,2023-04-28 00:00:00+00:00,351.32,151372,13714917990.0,2.295825,0.932581,2.58,10.911433,8.805013,20.788227,9.07222,2.69812,4655,11,28
2,POOL.OQ,2023-04-27 00:00:00+00:00,342.09,134656,13354594942.5,2.161033,0.932581,2.58,10.820371,8.573684,18.23804,9.436726,2.961625,4380,11,28
3,POOL.OQ,2023-04-26 00:00:00+00:00,332.25,126815,12970458562.5,2.098872,0.932389,2.58,10.50913,8.327068,17.713434,9.690015,-1.386086,4486,12,34
4,POOL.OQ,2023-04-25 00:00:00+00:00,336.92,151432,13152767190.0,2.128374,0.93248,2.58,10.656843,8.44411,17.962409,9.568132,-3.80036,4842,10,28
5,POOL.OQ,2023-04-24 00:00:00+00:00,350.23,201220,13672366297.5,2.212455,0.935852,2.58,11.077841,8.777694,18.672012,9.236992,2.036476,5742,10,28
6,POOL.OQ,2023-04-21 00:00:00+00:00,343.24,248200,13411697290.32,2.170273,0.932946,2.58,10.856746,8.602506,18.29935,9.4002,3.264238,8013,10,28
7,POOL.OQ,2023-04-20 00:00:00+00:00,332.39,371069,12987746365.02,2.10167,0.932414,2.58,10.513559,8.58667,17.720898,9.678324,-1.854312,9533,10,28
8,POOL.OQ,2023-04-19 00:00:00+00:00,338.67,158777,13233129942.059999,2.141378,0.932772,1.82,10.712196,8.748902,18.055707,9.515373,-0.07966,5281,12,34
9,POOL.OQ,2023-04-18 00:00:00+00:00,338.94,191369,13243679872.92,2.143085,0.932772,1.82,10.720736,8.755877,18.070102,9.50849,0.4475,6129,15,39


In [9]:
df.isna().sum()

Instrument                                                            0
Date                                                               5533
Price Close                                                        5533
Volume                                                             5533
Company Market Capitalization                                      5533
Price To Sales Per Share (Daily Time Series Ratio)                 5533
Beta                                                                 24
Earnings Per Share - Actual                                        5568
Price To Book Value Per Share (Daily Time Series Ratio)            5533
Forward Price To Book Value Per Share (Daily Time Series Ratio)    6013
P/E (Daily Time Series Ratio)                                      6287
Total Debt To Enterprise Value (Daily Time Series Ratio)           5553
Total Return                                                       5533
Number of Trades                                                

In [10]:
df = df.dropna()

In [11]:
df.to_csv("Dataframes/refinitiv.csv", index=False)